# UFO

<font color='#660066'>
   
### Table of Contents

- [Data](#Data)
    - [Scraping from UFORC](#Scraping-from-UFORC)
    - [Data Format](#data_format)
    - [Cleaing Data](#cleaning_pdata)
- [Basic anlayis](#Basic-Anylsis)
- [Milestone 3](#milestones)

In [1]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from lib.scraping import fetch_shape_urls,  \
                     fetch_all_sighting_urls, \
                     store_to_json, \
                     build_report_dataframe
from lib.postprocessing import get_time_of_occurrence, \
                           to_datetime_add_year, \
                           replace_empty_with_nan, \
                           distribution_histogram 
from lib.cleaning import clean_data, \
                        clean_duration_get_min
from lib.nuforc_analysis import *

In [2]:
data_folder = "data/"

file_url_list = data_folder+'full_list_of_urls.json'

report_file = 'jurgis_full.csv'

<font color='#660066'>
    
## Loading data

of course we dont want so scrape every time

In [3]:
df_reports = pd.read_csv(data_folder+report_file)
df_reports.sample(1).head()

,Unnamed: 0,Duration,Location,Occurred,Posted,Reported,Shape,Summary,url
22105,14216,45 minutes,"Myrtle Beach, SC",6/10/2014 22:30 (Entered as : 06/10/14 22:30),6/13/2014,6/10/2014 8:12:50 PM 20:12,Circle,"['Orange lights over water.', ""Sitting on 6th ...",http://www.nuforc.org/webreports/110/S110121.html


In [4]:
df_reports, df_madarr = clean_data(df_reports)

In [5]:
df_reports.head()

,Duration,Location,State,Occurred,Posted,Reported,Shape,Summary,nuforc_note,link
url,,,,,,,,,,
http://www.nuforc.org/webreports/143/S143627.html,10.000,Lutz,FL,2018-10-30 23:30:00,11/9/2018,2018-10-30 20:54:00,,['Blinking white light that moved every few se...,[((NUFORC Note: We have attempted to contact ...,[]
http://www.nuforc.org/webreports/089/S89377.html,4.500,Bridgewater,NJ,2011-11-23 00:45:00,6/5/2012,2012-06-05 12:37:00,,['EXPLAINED: Bright lights in Somerset county ...,[((NUFORC Note: Witness elects to remain tota...,[http://www.nj.com/news/index.ssf/2011/11/brig...
http://www.nuforc.org/webreports/019/S19862.html,2.000,South Fork,CO,2001-10-04 14:00:00,3/21/2003,2001-10-15 13:27:00,Cigar,"['Shinny Cigar shape in sky on clear day.', 'D...",[],[]
http://www.nuforc.org/webreports/121/S121007.html,10.000,Hattiesburg,MS,2014-12-17 20:00:00,8/13/2015,2015-08-09 21:11:00,Triangle,['I saw a triangular UFO with red and white li...,[],[]
http://www.nuforc.org/webreports/116/S116071.html,0.125,LaBelle,FL,2014-12-17 17:19:00,12/18/2014,2014-12-17 18:21:00,Triangle,"['White Triangle in South Florida', 'My co-wor...",[],[]


In [16]:
!ls data/

all_sightings.json
full_list_of_urls.json
jurgis_10k.csv
jurgis_5k.csv
jurgis_5k.json
jurgis_full.csv
jurgis_sightings.json
jurgis_sightings_1.json
readme.txt
ua_list_ua.xls


In [27]:
df_population = pd.read_excel('data/PctUrbanRural_State.xls')
df_population_all = pd.read_excel('data/PctUrbanRural_County.xls')

In [88]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [122]:
def get_us_population_data(df, dict, density_amount = 30):
    """
    Takes an input of US census data on US counties, their land areas and populations
    :density_amount: Our filter at which we say whether a county is 'urban' or 'rural'
                    We use 30 as default, as that returns 80% population worth of counties
                    as urban, which is inline with US census data, see:
                    https://www.census.gov/geo/reference/ua/urban-rural-2010.html
    returns:
        - DataFrame with calculated density and boolean index to whether the county is 
        rural or not
    """
    df_density = df[['STATENAME','COUNTYNAME', 'POP_COU','AREA_COU']].copy()
    df_density = df_density.drop(df_density[df_density.STATENAME == 'Puerto Rico'].index)
    df_density.AREA_COU = df_density.AREA_COU/(1000000)
    df_density['density'] = df_density.POP_COU/df_density.AREA_COU
    df_density['urban'] = False
    df_density.urban[(df_density.density > density_amount)] = True
    df_density.replace({'STATENAME': dict})
    return df_density

In [123]:
df_density = get_us_population_data(df_population_all, us_state_abbrev)

In [124]:
df_density.POP_COU[(df_density.urban == True)].sum()

262347473

In [126]:
df_density[(df_density.urban == True)].count()

STATENAME     1066
COUNTYNAME    1066
POP_COU       1066
AREA_COU      1066
density       1066
urban         1066
dtype: int64